# Рекомендация тарифов

# Tariff recommendation.

Многие клиенты сотового оператора пользуются архивными тарифами. Оператор хочет создать систему, которая сможет анализировать поведение клиентов и предложить им новые 
тарифы: «Смарт» или «Ультра».
Нам доступны данные о поведение клиентов, которые уже используют эти тарифы. Необходимо построить модель классификации, способную подобрать нужный тариф.
Нужно построить модель с максимальным значением accuracy (не менее 75%).

Many customers of the mobile operator use archival tariffs. The operator wants to create a system that can analyze the behavior of customers and offer them new
tariffs: "Smart" or "Ultra".
We have access to data about the behavior of customers who are already using these tariffs. It is necessary to build a classification model that can select the desired tariff.
It is necessary to build a model with a maximum accuracy value (at least 75%).

### Параметры.
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

### Arguments.
- calls — number of calls,
- minutes — total duration of calls in minutes,
- messages — number of sms messages,
- mb_used - used Internet traffic in Mb,
- is_ultra - what tariff did you use during the month ("Ultra" - 1, "Smart" - 0).

# Оглавление.

- [Шаг 1. Изучение файла.](#Step_1)
- [Шаг 2. Разбиение на выборки.](#Step_2)
- [Шаг 3. Исследование моделей.](#Step_3)<br />
[Пункт A. Дерево.](#Step_4) <br />
[Пункт B. Случайный лес.](#Step_5) <br />
[Пункт C. Логистическая регрессия.](#Step_6) <br />
- [Шаг 3. Проверка модели на тестовой выборке.](#Step_7)<br />
[Пункт E. Дерево.](#Step_8) <br />
[Пункт F. Случайный лес.](#Step_9) <br />
[Пункт G. Логистическая регрессия.](#Step_10) <br />
- [Шаг 4. Проверка моделей на адекватность.](#Step_11) <br />

# Table of contents.

- [Step 1. Examine the file.](#Step_1)
- [Step 2. Dividing into samples.](#Step_2)
- [Step 3. Explore models.](#Step_3)<br />
[Item A. Tree.](#Step_4) <br />
[Item B. Random forest.](#Step_5) <br />
[Item C. Logistic regression.](#Step_6) <br />
- [Step 3. Checking the model on the test set.](#Step_7)<br />
[Item E. Tree.](#Step_8) <br />
[Item F. Random forest.](#Step_9) <br />
[Item G. Logistic regression.](#Step_10) <br />
- [Step 4. Checking models for adequacy.](#Step_11) <br />

<a id='Step_1'></a>
## 1. Изучение файла.

## 1. Examining the file.

Загружаем библиотеки.

Loading libraries.

In [265]:
import pandas as pd
import numpy as np
import math 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

Загружаем файл.

Uploading a file.

In [266]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [267]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [268]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Посмотрим на баланс классов.

Let's look at class balance.

In [269]:
df.groupby('is_ultra').count()

,calls,minutes,messages,mb_used
is_ultra,,,,
0,2229,2229,2229,2229
1,985,985,985,985


Классы не сбалансированы: "1" более чем в два раза меньше чем "0".

Classes are not balanced: "1" is more than half of "0".

В нашем распоряжении датасет из 3214 уже подготовленных объектов. Классы не сбалансированы. 

We have a dataset of 3214 already prepared objects. The classes are not balanced.

<a id='Step_2'></a>
## 2. Разделение данных на выборки

## 2. Dividing the data into samples

Разделим признаки.
- is_ultra - зависимая переменная
- остальные - регрессоры

Let's share the symptoms.
- is_ultra - dependent variable
- the rest are regressors

In [270]:
y = df['is_ultra']
X = df.drop(['is_ultra'], axis=1)

Разделим сет на обучающую, валидационную и тестовую выборки.
- Обучаяющая - для подгонки модели (60% от сета).
- Валидационная - для выбора лучшей модели (20% от сета).
- Тестовая - для финальной проверки качества (20% от сета).

We divide the set into training, validation and test sets.
- Educational - to fit the model (60% of the set).
- Validation - to select the best model (20% of the set).
- Test - for the final quality check (20% of the set).

In [271]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

Мы делили выборки в два этапа. На первом мы разделили в пропорции 80% на 20%. Затем нам нужно от 80% отделить еще 20% от первоначальных 100%. Но если просто изъять 20% из оставшихся 80%, то число получится меньше чем нужно, поэтому нам надо было расчитать какой процент от 80% объектов равен 20% от 100% объектов. Для этого мы делим 80% объектов на 20%. Получаем 4. То есть нам нужно взять от 100% четвертую часть, что равно 25%. Результат:

We divided the samples in two stages. At the first, we divided in proportion 80% by 20%. Then we need to separate another 20% from the original 100% from 80%. But if we simply remove 20% of the remaining 80%, then the number will turn out to be less than necessary, so we had to calculate what percentage of 80% of objects is equal to 20% of 100% of objects. To do this, we divide 80% of the objects by 20%. We get 4. That is, we need to take the fourth part from 100%, which is equal to 25%. Result:

In [272]:
print('X_train', len(X_train))
print('X_valid', len(X_valid))
print('X_test', len(X_test))

X_train 1928
X_valid 643
X_test 643


Теперь у нас есть шесть частей сета.
- Обучение: X_train, y_train.
- Валидация: X_valid, y_valid
- Тест: X_test, y_test

Now we have six parts of the set.
- Training: X_train, y_train.
- Validation: X_valid, y_valid
- Test: X_test, y_test

<a id='Step_3'></a>
## 3. Исследование модели. 

## 3. Study of the model.

<a id='Step_4'></a>
### Дерево.

### Tree.

Будем экспериментировать с глубиной дерева и критерием.

We will experiment with the depth of the tree and the criterion.

In [273]:
max_depth_list_tree = []
criterion_list_tree = [] 
result_list_tree = []
for max_depth in range(1,6):
    for criterion in ('gini', 'entropy'):
        model_tree = DecisionTreeClassifier(random_state=42, max_depth=max_depth, criterion=criterion)
        model_tree.fit(X_train, y_train)
        prediction = model_tree.predict(X_valid)
        result = accuracy_score(prediction, y_valid)
        max_depth_list_tree.append(max_depth)
        criterion_list_tree.append(criterion)
        result_list_tree.append(result)

trees = pd.DataFrame(data={'max_depth' : max_depth_list_tree, 'criterion' : criterion_list_tree, 'result' : result_list_tree})
trees.sort_values(by='result', ascending=False)

,max_depth,criterion,result
9,5,entropy,0.791602
5,3,entropy,0.786936
7,4,entropy,0.785381
6,4,gini,0.780715
2,2,gini,0.774495
3,2,entropy,0.774495
4,3,gini,0.774495
8,5,gini,0.771384
0,1,gini,0.741835
1,1,entropy,0.741835


Похоже, что лучше всего использовать дерево глубиной 5 по энтропийному критерию, если конечно, мы вообще выберем дерево в качетсве нашей модели.

It seems to be best to use an entropy-deep 5 tree, if we choose a tree at all as our model.

<a id='Step_5'></a>
### Деревянный ансамбль.

### Wooden ensemble.

Здесь будем менять глубину, критерий и количество деревьев.

Here we will change the depth, criterion and number of trees.

In [274]:
max_depth_list_forest = []
criterion_list_forest = [] 
n_estimators_list_forest = []
result_list_forest = []
for max_depth in range(1,6):
    for criterion in ('gini', 'entropy'):
        for n_estimators in range(10, 100, 10):
            model_forest = RandomForestClassifier(random_state=42, max_depth=max_depth, criterion=criterion, 
                                                  n_estimators=n_estimators)
            model_forest.fit(X_train, y_train)
            prediction = model_forest.predict(X_valid)
            result = accuracy_score(prediction, y_valid)
            max_depth_list_forest.append(max_depth)
            criterion_list_forest.append(criterion)
            n_estimators_list_forest.append(n_estimators)
            result_list_forest.append(result)
forest = pd.DataFrame(data={'max_depth' : max_depth_list_forest, 'criterion' : criterion_list_forest, 
                            'n_estimators' : n_estimators_list_forest, 'result' : result_list_forest})
forest.sort_values(by='result', ascending=False).head()

,max_depth,criterion,n_estimators,result
64,4,entropy,20,0.805599
80,5,gini,90,0.804044
55,4,gini,20,0.804044
57,4,gini,40,0.804044
78,5,gini,70,0.804044


20 энтропийных деревьев глубиной 4 показывают лучший результат.

20 entropy trees with a depth of 4 show the best result.

<a id='Step_6'></a>
### Логистическая регрессия.

### Logistic regression.

In [275]:
model_logit = LogisticRegression(class_weight=None)
model_logit.fit(X_train, y_train)
prediction = model_logit.predict(X_valid)
result = accuracy_score(prediction, y_valid)
print(result)

0.7216174183514774


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Плоховато.

Too bad.

От чего бы нам не отмасштабировать параметры?

Why don't we scale the parameters?

In [276]:
from sklearn import preprocessing
X_train_scaled = preprocessing.scale(X_train)
X_valid_scaled = preprocessing.scale(X_valid)

In [277]:
model_logit_scale = LogisticRegression(C = 1e9, class_weight = {1 : 0.8}, solver='lbfgs')
model_logit_scale.fit(X_train_scaled, y_train)
prediction_logit_scale = model_logit_scale.predict(X_valid_scaled)
result = accuracy_score(prediction_logit_scale, y_valid)
print(result)

0.7511664074650077


Ого, мы вытянули регрессию за 75. Ура! (парметры подбирал руками). 

Wow, we pulled out the regression for 75. Hooray! (parameters selected by hand).

Посмотрим, где ошибается регрессия регрессия. 

Let's see where regression regression goes wrong.

In [278]:
y_valid_df = pd.DataFrame({'is_ultra' : y_valid})
y_valid_df['default_index'] = y_valid_df.index
y_valid_df.reset_index(drop=True, inplace=True)
y_valid_df.head()
wrong = []
for i in range(len(prediction)):
    if prediction_logit_scale[i] != y_valid_df['is_ultra'][i]:
        wrong.append(y_valid_df['default_index'][i])
df.iloc[wrong].groupby('is_ultra').count()

,calls,minutes,messages,mb_used
is_ultra,,,,
0,8,8,8,8
1,152,152,152,152


Что не так с классом "1". В чем причина, я не знаю. 

What is wrong with class "1". What is the reason, I don't know.

А давайте и главную компоненту запихаем (прости господи). 

And let's stuff the main component (God forgive me).

In [279]:
X_train_scaled_pca = X_train_scaled.copy()
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
X_train_scaled_pca = pca.fit_transform(X_train_scaled_pca)
X_valid_scaled_pca = X_valid_scaled.copy()
X_valid_scaled_pca = pca.fit_transform(X_valid_scaled_pca)

In [280]:
model_logit_scale = LogisticRegression(class_weight = {1 : 1.1}, solver='lbfgs')
model_logit_scale.fit(X_train_scaled_pca, y_train)
prediction = model_logit_scale.predict(X_valid_scaled_pca)
result = accuracy_score(prediction, y_valid)
print(result)

0.7542768273716952


Стало еще немного лучше, но коэффициенты интерпретировать уже не выйдет. В общем, поигрались и хватит. 

It has become a little better, but the coefficients will no longer be interpreted. In general, play around and that's enough.

### Вывод.
Все три модели показали достаточный уромень accuracy. Все три проверим на тестовой выборке.

### Conclusion.
All three models showed a sufficient level of accuracy. We will check all three on a test sample.

<a id='Step_7'></a>
## 4. Проверим модель на тестовой выборке.

## 4. Check the model on the test set.

<a id='Step_8'></a>
### Дерево (глубины 5 и энтропийным критерием).

### Tree (depth 5 and entropy criterion).

In [281]:
model_tree_test = DecisionTreeClassifier(random_state=42, max_depth=5, criterion='entropy')
model_tree_test.fit(X_train, y_train)
prediction_tree_test = model_tree_test.predict(X_test)
result_tree_test = accuracy_score(prediction_tree_test, y_test)
print(result_tree_test)

0.7993779160186625


<a id='Step_19'></a>
### Случайный лес (глубины 4, 20-ю деревьями и энтропийным критерием)

### Random forest (depth 4, 20 trees and entropy criterion)

In [282]:
model_forest_test = RandomForestClassifier(random_state=42, max_depth=4, criterion='entropy', n_estimators=20)
model_forest_test.fit(X_train, y_train)
prediction_forest_test = model_forest_test.predict(X_test)
result_forest_test = accuracy_score(prediction_forest_test, y_test)
print(result_forest_test)

0.8118195956454122


<a id='Step_10'></a>
### Логистическая регрессия (отмасштабированые параметры, балансированная, регуляризация отключена, дефолтный solver). 

### Logistic regression (scaled parameters, balanced, regularization off, default solver).

In [283]:
X_test_scaled = preprocessing.scale(X_test)
model_logit_scale = LogisticRegression(C = 1e9, class_weight = {1 : 0.8}, solver='lbfgs')
model_logit_scale.fit(X_train_scaled, y_train)
prediction_logit_scale = model_logit_scale.predict(X_test_scaled)
result = accuracy_score(prediction_logit_scale, y_test)
print(result)

0.7573872472783826


### Вывод.
- Мы добились необходимого результата всеми тремя моделями.
- Хуже всех показала себя регрессия. Надо разбираться.
- Оптимальным решением, из представленных моделей, являеется дерево. Оно быстрее всех и не сильно усутпает случайному лесу.

### Conclusion.
- We have achieved the desired result with all three models.
- Worst of all showed itself regression. Need to figure it out.
- The optimal solution, from the presented models, is a tree. It is the fastest and is not much inferior to the random forest.